In [ ]:
from pyspark.sql.functions import lit, col, current_timestamp, max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import datetime

In [ ]:
def tables_of_schema(catalog_name):
    table_name_df = spark.sql(f"""
        SELECT table_name
        FROM zz_bronze_{catalog_name}.information_schema.tables
        WHERE table_schema = 'default'
        ORDER BY table_name
    """)

    table_list = [row['table_name'] for row in table_name_df.collect()]
    return table_list

In [ ]:
integrity_schema = StructType([
                StructField('nome_schema', StringType(), True),
                StructField('nome_table', StringType(), True),
                StructField('qtd_raw', IntegerType(), True),
                StructField('qtd_bronze', IntegerType(), True),
                StructField('nome_catalog', StringType(), True)
            ])

integrity_df = spark.createDataFrame([], integrity_schema)

catalog_list = ['area_1', 'area_2', 'area_3']

for catalog in catalog_list:
    table_list = tables_of_schema(catalog)
    for table in table_list:
        df = spark.table(f'zz_bronze_{catalog}.default.{table}')
        df = df.withColumn('nome_catalog', lit(catalog))

        integrity_df = integrity_df.union(df)

integrity_df = integrity_df.withColumn('data', current_timestamp())
integrity_df.write.mode('append').saveAsTable('controle.integridade.integridade_raw_bronze')
